---
# **PREDIÇÃO DE TEMPERATURA**
---


# 📌 1. Entendimento do problema


Objetivo: entender o comportamento da temperatura média diária e identificar padrões ou variáveis associadas úteis para modelagem preditiva.

Contexto:
- Previsão de temperatura é essencial para agricultura, saúde pública e planejamento energético.
- Este projeto foca em prever temperatura diária com base em variáveis meteorológicas passadas (ex: umidade, precipitação, vento).

Referências:
- Modelos comuns: Regressão Linear, Random Forest, SVM, XGBoost, LSTM.
- Features típicas: temperatura defasada, umidade relativa, precipitação, variáveis temporais (dia, mês, estação).





In [34]:
# Pacotes principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.features import build_features


sns.set(style="whitegrid")

ModuleNotFoundError: No module named 'src'

In [6]:
import os
print(os.getcwd())

c:\Users\laiss\Projetos\modelagem_climatica\predicting-temperature\notebooks


# 📌 2. Carregamento dos Dados


In [ ]:

df = pd.read_csv("data/raw/nasa_rio_vermelho.csv")
df = build_features(df)
df.head()

In [23]:
path = "C:/Users/laiss/Projetos/modelagem_climatica/predicting-temperature/data/raw/nasa_rio_vermelho.csv"
df = pd.read_csv(path, skiprows=26)  # a NASA inclui 10 linhas de metadados
df.head()

,YEAR,DOY,T2M,RH2M,PRECTOTCORR,WS2M,ALLSKY_SFC_SW_DWN
0,2020,1,25.97,87.48,2.92,5.24,19.97
1,2020,2,25.07,84.72,10.36,8.34,17.31
2,2020,3,24.60,80.82,3.49,8.01,24.51
3,2020,4,24.51,81.58,3.91,6.56,12.47
4,2020,5,24.62,80.98,0.66,3.83,27.13


# 📌 3. QUALIDADE DOS DADOS

In [31]:
# Verifica tipos, nulos, e duplicatas
df.info()
df.isnull().sum()
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1819 entries, 2020-01-08 to 2024-12-30
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 1819 non-null   int64  
 1   day_of_year          1819 non-null   int64  
 2   temperature          1819 non-null   float64
 3   humidity             1819 non-null   float64
 4   precipitation        1819 non-null   float64
 5   wind_speed           1819 non-null   float64
 6   irradiance           1819 non-null   float64
 7   month                1819 non-null   int32  
 8   temperature_lag1     1819 non-null   float64
 9   temperature_lag3     1819 non-null   float64
 10  temperature_lag7     1819 non-null   float64
 11  temperature_roll3    1819 non-null   float64
 12  humidity_roll3       1819 non-null   float64
 13  precipitation_roll7  1819 non-null   float64
 14  target               1819 non-null   float64
dtypes: float64(12), int3

np.int64(0)

# 📌 4. ENGENHARIA DE VARIÁVEIS

In [27]:
# Criação de variáveis temporais
df["month"] = df.index.month

# Lags de temperatura
df['temperature_lag1'] = df['temperature'].shift(1)
df['temperature_lag3'] = df['temperature'].shift(3)
df['temperature_lag7'] = df['temperature'].shift(7)

# Rolling mean
df['temperature_roll3'] = df['temperature'].rolling(window=3).mean()
df['humidity_roll3'] = df['humidity'].rolling(window=3).mean()
df['precipitation_roll7'] = df['precipitation'].rolling(window=7).sum()

df.tail()

,year,day_of_year,temperature,humidity,precipitation,wind_speed,irradiance,month,temperature_lag1,temperature_lag3,temperature_lag7,temperature_roll3,humidity_roll3,precipitation_roll7
date,,,,,,,,,,,,,,
2024-12-27,2024,362,23.60,75.40,0.07,2.22,30.80,12,22.40,22.97,24.33,23.236667,79.846667,15.33
2024-12-28,2024,363,24.24,77.27,0.16,2.32,27.95,12,23.60,23.71,24.29,23.413333,77.383333,15.14
2024-12-29,2024,364,23.45,79.46,0.29,3.94,28.86,12,24.24,22.40,23.43,23.763333,77.376667,7.49
2024-12-30,2024,365,22.94,83.54,0.84,3.42,17.70,12,23.45,23.60,22.18,23.543333,80.090000,7.91
2024-12-31,2024,366,23.76,86.45,1.18,4.71,25.51,12,22.94,24.24,22.97,23.383333,83.150000,8.45


In [28]:
# 📌 Definição da variável-alvo (target)

# Exemplo: queremos prever a temperatura média do dia seguinte
df['target'] = df['temperature'].shift(-1)

# Remove últimas linhas com NaN no target
df = df.dropna()

In [29]:
df.head()

,year,day_of_year,temperature,humidity,precipitation,wind_speed,irradiance,month,temperature_lag1,temperature_lag3,temperature_lag7,temperature_roll3,humidity_roll3,precipitation_roll7,target
date,,,,,,,,,,,,,,,
2020-01-08,2020,8,24.54,85.94,23.25,5.65,14.69,1,26.06,24.62,25.97,25.356667,84.326667,49.54,26.04
2020-01-09,2020,9,26.04,83.84,0.14,2.85,28.47,1,24.54,25.47,25.07,25.546667,85.213333,39.32,26.29
2020-01-10,2020,10,26.29,90.91,6.33,4.37,22.46,1,26.04,26.06,24.60,25.623333,86.896667,42.16,26.21
2020-01-11,2020,11,26.21,91.71,25.27,6.02,16.31,1,26.29,24.54,24.51,26.180000,88.820000,63.52,25.29
2020-01-12,2020,12,25.29,78.31,1.98,6.09,23.39,1,26.21,26.04,24.62,25.930000,86.976667,64.84,24.88


In [30]:
df.to_csv("../data/processed/base_modelagem.csv", index=False)


# 📌 5. ANÁLISE EXPLORATÓRIA

In [ ]:
# Série temporal da temperatura média
plt.figure(figsize=(14, 4))
plt.plot(df["date"], df["temp_mean"])
plt.title("Temperatura Média Diária")
plt.xlabel("Data")
plt.ylabel("Temp (°C)")
plt.tight_layout()
plt.show()

In [ ]:
# Histogramas por mês
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x="month", y="temp_mean")
plt.title("Distribuição da temperatura média por mês")
plt.xlabel("Mês")
plt.ylabel("Temp (°C)")
plt.show()


In [ ]:
# Correlação entre variáveis numéricas
plt.figure(figsize=(10, 8))
corr = df.select_dtypes(include=np.number).corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Mapa de correlação entre variáveis")
plt.show()

# 📌 6. INSIGHTS

- A temperatura apresenta forte sazonalidade anual.
- Há correlação significativa entre temperatura média e variáveis como umidade e precipitação.
- Lags e médias móveis parecem úteis para modelos supervisionados.

Próximos passos:
- Construção de modelos simples para baseline (regressão linear, RF)
- Comparação com modelos não-lineares e sequenciais